In [40]:
import os
import time
import gzip
import shutil
import subprocess
import tkinter as tk
from PIL import Image
from pathlib import Path
import customtkinter as ctk
from tkinter import ttk, filedialog

In [18]:

class App(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.colmode = "light"
        self.colswitch = "#eceff1"
        self.update_mode()

        self.marpledir = os.path.join(os.path.join(Path.home(), 'marple'))
        
        self.geometry("600x920")
        self.title("MARPLE")
        self.attributes('-alpha', 0.95)
        
        self.font = ctk.CTkFont(family="Helvetica", size=16)
        self.large_font = ctk.CTkFont(family="Helvetica", size=20)
        
        self.barcode_rows = []
        self.transfer_type = 'Pgt'
        
        # Image setup
        logo_path = os.path.join(os.getcwd(), "MARPLE_logo.png")
        try:
            logo_image = ctk.CTkImage(Image.open(logo_path), size=(600, 200))
        except Exception as e:
            print(f"Error loading image: {e}")
            logo_image = None
        
        self.logo_label = ctk.CTkLabel(self, image=logo_image, text="", bg_color=self.colswitch)
        self.logo_label.pack(pady=(20, 10))
        
        # Notification frame
        self.notification_frame = ctk.CTkFrame(self, bg_color="gray10")
        
        # Run MARPLE button
        self.run_marple_button = ctk.CTkButton(self, text="RUN MARPLE", command=self.run_marple, corner_radius=1, font=self.large_font)
        self.run_marple_button.pack(pady=(30, 30))
        
        # Toggle transfer reads options button
        self.toggle_transfer_button = ctk.CTkButton(self, text="Toggle Transfer Reads Options", command=self.toggle_transfer_options, corner_radius=1, font=self.font)
        self.toggle_transfer_button.pack(pady=(20, 10))
        
        # Transfer reads options frame
        self.transfer_options_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        
        # Select directory button
        self.select_dir_button = ctk.CTkButton(self.transfer_options_frame, command=self.select_experiment, text="Select MinKNOW Directory", corner_radius=1, font=self.font)
        self.select_dir_button.pack(pady=(20, 10))
        
        # Experiment name label
        
        self.expname_label = ctk.CTkLabel(self.transfer_options_frame, text="Experiment Name: ", corner_radius=1, font=self.font)
        self.expname_label.pack(pady=(10, 20), fill="x")
        
        # Segmented button
        self.segmented_button = ctk.CTkSegmentedButton(self.transfer_options_frame, values=['Pgt', 'Pst'], command=self.on_segmented_button_click, corner_radius=1, font=self.font)
        self.segmented_button.set('Pgt')
        self.segmented_button.pack(pady=(10, 20))
        
        # Scrollable frame
        self.scrollable_frame = ctk.CTkFrame(self.transfer_options_frame, bg_color=self.colswitch)
        self.scrollable_frame.pack(fill="both", expand=True)

        self.canvas = ctk.CTkCanvas(self.scrollable_frame, bg=self.colswitch)
        self.scrollbar = ctk.CTkScrollbar(self.scrollable_frame, orientation="vertical", command=self.canvas.yview, corner_radius=1)

        self.canvas.pack(side="left", fill="both", expand=True)
        self.scrollbar.pack(side="right", fill="y")

        self.frame = ctk.CTkFrame(self.canvas, bg_color=self.colswitch)
        self.canvas.create_window((0, 0), window=self.frame, anchor="nw")

        self.canvas.config(yscrollcommand=self.scrollbar.set)
        self.frame.bind("<Configure>", self.on_frame_configure)

        # Configure the parent frame to expand and fill the available space
        self.frame.grid_columnconfigure(0, weight=1)
 
        # Add barcode row button
        self.add_row_button = ctk.CTkButton(self.transfer_options_frame, text="Add Barcode Row", command=self.add_barcode_row, corner_radius=1, font=self.font)
        self.add_row_button.pack(pady=(10, 20))
        
        # Transfer reads button
        self.transfer_reads_button = ctk.CTkButton(self.transfer_options_frame, command=self.transfer_reads, text="Transfer Reads", corner_radius=1, font=self.large_font)
        self.transfer_reads_button.pack(pady=(10, 20))
        
        self.add_barcode_row()
        
        # Mode switch button
        self.mode_switch_button = ctk.CTkButton(self, text=f'{"Dark mode" if self.colmode == "light" else "Light mode"}', command=self.switch_mode, corner_radius=1, font=self.font)
        self.mode_switch_button.pack(pady=(10, 20))

        # Progress bar
        self.progress_bar = ctk.CTkProgressBar(self, orientation="horizontal", mode="indeterminate")
        self.progress_bar.pack(pady=(10, 20))

    def update_mode(self):
        ctk.set_appearance_mode(self.colmode)
        if self.colmode == "light":
            self.colswitch = "#eceff1"
        else:
            self.colswitch = "gray10"

    def switch_mode(self):
        self.colmode = "dark" if self.colmode == "light" else "light"
        self.update_mode()
        self.update_ui()
        self.mode_switch_button.configure(text=f'{"Dark mode" if self.colmode == "light" else "Light mode"}')

    def update_ui(self):
        self.logo_label.configure(bg_color=self.colswitch)
        self.scrollable_frame.configure(bg_color=self.colswitch)
        self.canvas.configure(bg=self.colswitch)
        self.frame.configure(bg_color=self.colswitch)
        for row in self.barcode_rows:
            row[0].configure(bg_color=self.colswitch)
            row[1].configure(bg_color=self.colswitch)
        self.notification_frame.configure(bg_color=self.colswitch)
        self.transfer_options_frame.configure(bg_color=self.colswitch)

    def add_barcode_row(self):
        row = ctk.CTkFrame(self.frame, bg_color=self.colswitch, corner_radius=1)
        row.grid(row=len(self.barcode_rows), column=0,  sticky="ew")
        
        barcode_label = ctk.CTkLabel(row, text="barcode", font=self.font)
        barcode_label.grid(row=0, column=0, padx=5, sticky="ew")
        
        barcode_entry = ctk.CTkEntry(row, width=60, corner_radius=1, font=self.font)
        barcode_entry.grid(row=0, column=1,  sticky="ew")
        
        sample_entry = ctk.CTkEntry(row, width=300, corner_radius=1, font=self.font)
        sample_entry.grid(row=0, column=2, padx=20, sticky="ew")
        
        # Configure the columns of the row frame to expand and fill the available space
        row.grid_columnconfigure(0, weight=1)
        row.grid_columnconfigure(1, weight=1)
        row.grid_columnconfigure(2, weight=1)
        
        self.barcode_rows.append((barcode_entry, sample_entry))
    
    def on_frame_configure(self, event):
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))
    
    def run_marple(self):
        run_cmd = "snakemake --cores all --rerun-incomplete"
        log_file = "snakemake.log"

        if shutil.which("mamba") is not None:
            try:
                env_list = subprocess.check_output(["mamba", "env", "list"], text=True)
                if "marple-env" in env_list:
                    self.printin("Running MARPLE...")

                    # Unlock snakemake
                    subprocess.Popen(["mamba", "run", "-n", "marple-env", "snakemake", "--unlock"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=self.marpledir)

                    # Run snakemake command
                    with open(log_file, "w") as log:
                        process = subprocess.Popen(["mamba", "run", "-n", "marple-env", "snakemake", "--cores", "all", "--rerun-incomplete"], cwd=self.marpledir, stdout=log, stderr=log)
                    
                    self.progress_bar.start()
                    self.check_process(process, log_file)
                else:
                    self.printin("mamba environment marple-env not found.")
            except subprocess.CalledProcessError as e:
                self.printin(f"Error running command: {e}")
        else:
            self.printin("mamba not found.")

    def check_process(self, process, log_file):
        if process.poll() is None:
            self.after(100, self.check_process, process, log_file)
        else:
            self.progress_bar.stop()
            status = process.returncode
            if status != 0:
                self.printin(f"Command failed with exit status {status}. Error log:")
                with open(log_file, "r") as log:
                    self.printin(log.read())
            else:
                self.printin("MARPLE run completed successfully.")

    def printin(self, stdout):
        self.notification_frame.pack(side="bottom", fill="x")
        for widget in self.notification_frame.winfo_children():
            widget.destroy()
        
        label = ctk.CTkLabel(self.notification_frame, text=stdout, bg_color=self.colswitch)
        label.pack()
        
        self.after(5000, self.clear_notification)
    
    def clear_notification(self):
        for widget in self.notification_frame.winfo_children():
            widget.destroy()
        self.notification_frame.pack_forget()
    
    def select_experiment(self):
        default_dir = "/var/lib/minknow/data/"
        
        self.minknow_dir = filedialog.askdirectory(initialdir=default_dir)
        if self.minknow_dir:
            self.printin(f"Selected Experiment: {self.minknow_dir}")
            expname = os.path.basename(self.minknow_dir)
            self.expname_label.configure(text=f"Experiment Name: {expname}")

    def transfer_reads(self):
        if not self.minknow_dir:
            self.printin("MinKNOW directory not selected.")
            return
        
        self.progress_bar.start()
        self.after(100, self._transfer_reads)

    def _transfer_reads(self):
        try:
            for barcode_entry, sample_entry in self.barcode_rows:
                barcode = f'barcode{barcode_entry.get().strip()}'
                sample = sample_entry.get().strip()
                
                if not barcode:
                    continue  # Skip empty barcode entries
                
                for root, dirs, files in os.walk(self.minknow_dir):
                    for dir in dirs:
                        if dir == 'pass':
                            barcode_dir = os.path.join(root, dir, barcode)
                            try:
                                output_file = os.path.join(self.marpledir, 'reads', self.transfer_type.lower(), f'{sample}.fastq.gz')
                                with gzip.open(output_file, 'wb') as fout:
                                    for file in os.listdir(barcode_dir):
                                        if file.endswith(".gz"):
                                            with gzip.open(os.path.join(barcode_dir, file), 'rb') as f:
                                                reads = f.readlines()
                                                fout.writelines(reads)
                                        elif file.endswith(".fastq"):
                                            with open(os.path.join(barcode_dir, file), 'rb') as f:
                                                reads = f.readlines()
                                                fout.writelines(reads)
                                self.printin(f"Successfully transferred reads for barcode {barcode} to {output_file}")
                            except Exception as e:
                                self.printin(f"An error occurred while processing barcode {barcode}: {e}")                
        except Exception as e:
            self.printin(f"An error occurred: {e}")
        finally:
            self.progress_bar.stop()
            
    def on_segmented_button_click(self, choice):
        self.transfer_type = choice

    def toggle_transfer_options(self):
        if self.transfer_options_frame.winfo_ismapped():
            self.transfer_options_frame.pack_forget()
        else:
            self.transfer_options_frame.pack(fill="both", expand=True)

app = App()
app.mainloop()

In [40]:
import tkinter as tk
from tkinter import Menu, filedialog, ttk
import customtkinter as ctk
import os
import gzip
import shutil
import subprocess
from pathlib import Path
from PIL import Image
import threading

class App(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.colmode = "light"
        self.colswitch = "#eceff1"
        self.update_mode()

        self.marpledir = os.path.join(os.path.join(Path.home(), 'marple'))
        
        self.geometry("600x920")
        self.title("MARPLE")
        self.attributes('-alpha', 0.95)
        
        self.font = ctk.CTkFont(family="Helvetica", size=16)
        self.large_font = ctk.CTkFont(family="Helvetica", size=20)
        
        self.barcode_rows = []
        self.transfer_type = 'Pgt'
        self.minknow_dir = ""

        # Image setup
        logo_path = os.path.join(os.getcwd(), "MARPLE_logo.png")
        try:
            logo_image = ctk.CTkImage(Image.open(logo_path), size=(600, 200))
        except Exception as e:
            print(f"Error loading image: {e}")
            logo_image = None
        
        self.logo_label = ctk.CTkLabel(self, image=logo_image, text="", bg_color=self.colswitch)
        self.logo_label.pack(pady=(20, 10))

        # Menu setup (Sandwich Menu)
        self.menu_bar = Menu(self)
        self.config(menu=self.menu_bar)

        self.marple_menu = Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Menu", menu=self.marple_menu)
        self.marple_menu.add_command(label="Run MARPLE", command=self.show_home)
        self.marple_menu.add_command(label="Transfer Reads", command=self.show_transfer_reads)
        self.marple_menu.add_command(label="Results", command=self.show_results)
        self.marple_menu.add_command(label="About", command=self.show_about)
        self.marple_menu.add_separator()
        self.marple_menu.add_command(label="Exit", command=self.quit)

        # Light/Dark mode toggle in the menu
        self.theme_menu = Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Settings", menu=self.theme_menu)
        self.theme_menu.add_command(label=f'{"Dark mode" if self.colmode == "light" else "Light mode"}', command=self.switch_mode)

        # Run MARPLE button (Home Page)
        self.run_marple_button = ctk.CTkButton(self, text="RUN MARPLE", command=self.run_marple, corner_radius=1, font=self.large_font)
        self.run_marple_button.pack(pady=(30, 30))

        # Placeholder for dynamic content
        self.dynamic_frame = None

        # Progress bar
        self.progress_bar = ctk.CTkProgressBar(self, orientation="horizontal")
        self.progress_bar.pack(pady=(10, 20))
        
        # Notification frame
        self.notification_frame = ctk.CTkFrame(self, bg_color="gray10")

    def update_mode(self):
        ctk.set_appearance_mode(self.colmode)
        self.colswitch = "#eceff1" if self.colmode == "light" else "gray10"

    def switch_mode(self):
        self.colmode = "dark" if self.colmode == "light" else "light"
        self.update_mode()
        self.update_ui()
        # Update the theme toggle text in the menu
        self.theme_menu.entryconfig(0, label=f'{"Dark mode" if self.colmode == "light" else "Light mode"}')

    def update_ui(self):
        self.logo_label.configure(bg_color=self.colswitch)
        if self.dynamic_frame:
            self.dynamic_frame.configure(bg_color=self.colswitch)

    def show_home(self):
        self.clear_dynamic_frame()
        self.run_marple_button.pack(pady=(30, 30))

    def show_transfer_reads(self):
        self.clear_dynamic_frame()
        self.dynamic_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        self.dynamic_frame.pack(fill="both", expand=True, pady=20)

        # Transfer reads options
        self.select_dir_button = ctk.CTkButton(self.dynamic_frame, command=self.select_experiment, text="Select MinKNOW Directory", corner_radius=1, font=self.font)
        self.select_dir_button.pack(pady=(20, 10))

        self.expname_label = ctk.CTkLabel(self.dynamic_frame, text="Experiment Name: ", corner_radius=1, font=self.font)
        self.expname_label.pack(pady=(10, 20))

        self.segmented_button = ctk.CTkSegmentedButton(self.dynamic_frame, values=['Pgt', 'Pst'], command=self.on_segmented_button_click, corner_radius=1, font=self.font)
        self.segmented_button.set('Pgt')
        self.segmented_button.pack(pady=(10, 20))

        self.add_row_button = ctk.CTkButton(self.dynamic_frame, text="Add Barcode Row", command=self.add_barcode_row, corner_radius=1, font=self.font)
        self.add_row_button.pack(pady=(10, 20))

        self.transfer_reads_button = ctk.CTkButton(self.dynamic_frame, command=self.transfer_reads, text="Transfer Reads", corner_radius=1, font=self.large_font)
        self.transfer_reads_button.pack(pady=(10, 20))

        # Create a frame with a scrollbar
        self.canvas = ctk.CTkCanvas(self.dynamic_frame, bg=self.colswitch)
        self.scrollbar = ttk.Scrollbar(self.dynamic_frame, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ctk.CTkFrame(self.canvas, bg_color=self.colswitch, corner_radius=1)

        # Configure the canvas and scrollbar
        self.scrollable_frame.bind("<Configure>", lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        # Limit the height of the canvas (5 rows height)
        self.canvas.pack(side="left", fill="both", expand=True, padx=(5, 0), pady=20)
        self.scrollbar.pack(side="right", fill="y")

    def add_barcode_row(self):
        row = ctk.CTkFrame(self.scrollable_frame, bg_color=self.colswitch, corner_radius=1)
        row.pack(fill="x", padx=5, pady=2)
        row.grid_columnconfigure(0, weight=1)

        barcode_label = ctk.CTkLabel(row, text="Barcode:", font=self.font)
        barcode_label.pack(side="left", padx=5)

        barcode_entry = ctk.CTkEntry(row, width=60, corner_radius=1, font=self.font)
        barcode_entry.pack(side="left", padx=5)

        sample_label = ctk.CTkLabel(row, text="Sample Name:", font=self.font)
        sample_label.pack(side="left", padx=5)

        sample_entry = ctk.CTkEntry(row, width=200, corner_radius=1, font=self.font)
        sample_entry.pack(side="left", padx=5)

        self.barcode_rows.append((barcode_entry, sample_entry))
        
        # Enable scrolling if more than 5 rows
        if len(self.barcode_rows) > 5:
            self.canvas.configure(height=5 * 40)  # Each row is about 40 pixels high

    def show_about(self):
        self.clear_dynamic_frame()
        self.dynamic_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        self.dynamic_frame.pack(fill="both", expand=True, pady=20)

        about_label = ctk.CTkLabel(self.dynamic_frame, text="MARPLE Diagnostics:\npoint-of-care, strain-level disease diagnostics and\nsurveillance tool for complex fungal pathogens\n\nVersion: 2.0-alpha", font=self.large_font)
        about_label.pack(pady=(20, 20))

        devs_label = ctk.CTkLabel(self.dynamic_frame, text="Software & Legacy Code Developers:\nLoizos Savva\nAnthony Bryan\nGuru V. Radhakrishnan")
        devs_label.pack(pady=(20, 20))
        
        copyright_label = ctk.CTkLabel(self.dynamic_frame, text="© 2024 Saunders Lab")
        copyright_label.pack()

    def clear_dynamic_frame(self):
        if self.dynamic_frame:
            self.dynamic_frame.destroy()
        self.run_marple_button.pack_forget()

    def on_segmented_button_click(self, choice):
        self.transfer_type = choice

    def select_experiment(self):
        default_dir = "/var/lib/minknow/data/"
        self.minknow_dir = filedialog.askdirectory(initialdir=default_dir)
        if self.minknow_dir:
            expname = os.path.basename(self.minknow_dir)
            self.expname_label.configure(text=f"Experiment Name: {expname}")

    def transfer_reads(self):
        if not self.minknow_dir:
            print("MinKNOW directory not selected.")
            return

        # Start the progress bar
        self.progress_bar.configure(mode="indeterminate")
        self.progress_bar.start()

        # Run the task in a separate thread to avoid freezing the UI
        thread = threading.Thread(target=self.process_reads)
        thread.start()

    def process_reads(self):
        try:
            total_barcodes = len(self.barcode_rows)
            if total_barcodes == 0:
                self.printin("No barcodes to process.")
                self.progress_bar.stop()
                return

            for barcode_entry, sample_entry in self.barcode_rows:
                barcode = format(int(barcode_entry.get().strip()), '02d')
                sample = sample_entry.get().strip()

                if not barcode:
                    continue  # Skip empty barcode entries

                for root, dirs, files in os.walk(self.minknow_dir):
                    for dir in dirs:
                        if dir == 'pass':
                            barcode_dir = os.path.join(root, dir, f'barcode{barcode}')
                            try:
                                output_file = os.path.join(self.marpledir, 'reads', self.transfer_type.lower(), f'{sample}.fastq.gz')
                                with gzip.open(output_file, 'wb') as fout:
                                    for file in os.listdir(barcode_dir):
                                        if file.endswith(".gz"):
                                            with gzip.open(os.path.join(barcode_dir, file), 'rb') as f:
                                                reads = f.readlines()
                                                fout.writelines(reads)
                                        elif file.endswith(".fastq"):
                                            with open(os.path.join(barcode_dir, file), 'rb') as f:
                                                reads = f.readlines()
                                                fout.writelines(reads)
                                self.printin(f"Successfully transferred reads for barcode{barcode} to {output_file}")
                            except Exception as e:
                                self.printin(f"An error occurred while processing barcode{barcode}: {e}")
        finally:
            self.progress_bar.stop()

    def run_marple(self):
        # Start progress bar
        self.progress_bar.configure(mode="indeterminate")
        self.progress_bar.start()

        # Run the Snakemake process in a separate thread
        thread = threading.Thread(target=self.run_snakemake)
        thread.start()

    def run_snakemake(self):
        
        log_file = "snakemake.log"

        if shutil.which("mamba") is not None:
            try:
                env_list = subprocess.check_output(["mamba", "env", "list"], text=True)
                if "marple-env" in env_list:
                    # self.printin("Running MARPLE...")

                    # if os.path.exists(os.path.join(self.marpledir,".snakemake", "locks", "0.input.lock")):
                    #     pass
                    # else:
                    #     subprocess.Popen(["mamba", "run", "-n", "marple-env", "snakemake", "--unlock"], 
                    #                     stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL, cwd=self.marpledir
                    #                     )

                    # Run snakemake command and get real-time output
                    with open(log_file, "w") as log:
                        process = subprocess.Popen(["mamba", "run", "-n", "marple-env", "snakemake", "--cores", "all", "--rerun-incomplete"],
                            cwd=self.marpledir, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1
                        )

                        self.display_output(process)  # Display real-time output
                else:
                    self.printin("mamba environment marple-env not found.")
            except subprocess.CalledProcessError as e:
                self.printin(f"Error running command: {e}")
        else:
            self.printin("mamba not found.")

        # Stop the progress bar
        self.progress_bar.stop()

    def display_output(self, process):
        if not hasattr(self, 'output_text'):
            self.output_text = ctk.CTkTextbox(self, width=600, height=400)
            self.output_text.pack(pady=(20, 20))
            self.output_text.configure(state="disabled")

        self.output_text.configure(state="normal")  # Enable the text box

        for line in process.stdout.readline():
            if line:
                self.output_text.insert("end", line)
                self.output_text.see("end")
            self.update()

        process.stdout.close()
        process.wait()

        if process.returncode != 0:
            self.output_text.insert("end", f"MARPLE run failed with exit code {process.returncode}. Error: {process.stderr.read()}")
            self.output_text.see("end")
        else:
            self.output_text.insert("end", "MARPLE run completed successfully.")
            self.output_text.see("end")

        self.output_text.configure(state="disabled")  # Disable the text box again

    def toggle_output_display(self):
        if self.output_text.winfo_viewable():
            self.output_text.pack_forget()
        else:
            self.output_text.pack(pady=(20, 20))
    
    def check_process(self, process, log_file):
        if process.poll() is None:
            self.after(100, self.check_process, process, log_file)
        else:
            self.progress_bar.stop()
            status = process.returncode
            if status != 0:
                self.printin(f"Command failed with exit status {status}. Error log:")
                with open(log_file, "r") as log:
                    self.printin(log.read())
            else:
                self.printin("MARPLE run completed successfully.")

    def printin(self, stdout):
        self.notification_frame.pack(side="bottom", fill="x", expand="True")
        for widget in self.notification_frame.winfo_children():
            widget.destroy()
        
        label = ctk.CTkLabel(self.notification_frame, text=stdout, bg_color=self.colswitch)
        label.pack()
        
        self.after(5000, self.clear_notification)
    
    def clear_notification(self):
        for widget in self.notification_frame.winfo_children():
            widget.destroy()
        self.notification_frame.pack_forget()

    def show_results(self):
        self.clear_dynamic_frame()
        self.dynamic_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        self.dynamic_frame.pack(fill="both", expand=True, pady=20)

        # Two columns
        self.dynamic_frame.grid_columnconfigure(0, weight=1)
        self.dynamic_frame.grid_columnconfigure(1, weight=1)

        # Pgt Column
        pgt_label = ctk.CTkLabel(self.dynamic_frame, text="Pgt", font=self.large_font, anchor="center")
        pgt_label.grid(row=0, column=0, pady=(10, 10))

        pgt_tree_button = ctk.CTkButton(self.dynamic_frame, text="Tree", command=lambda: self.open_file("pgt", "trees", "pgt_all.pdf"), corner_radius=1, font=self.font)
        pgt_tree_button.grid(row=1, column=0, pady=(10, 10))

        pgt_report_button = ctk.CTkButton(self.dynamic_frame, text="Report", command=lambda: self.open_file("pgt", "report", "pgt.multiqc.html"), corner_radius=1, font=self.font)
        pgt_report_button.grid(row=2, column=0, pady=(10, 10))

        # Pst Column
        pst_label = ctk.CTkLabel(self.dynamic_frame, text="Pst", font=self.large_font, anchor="center")
        pst_label.grid(row=0, column=1, pady=(10, 10))

        pst_tree_button = ctk.CTkButton(self.dynamic_frame, text="Tree", command=lambda: self.open_file("pst", "trees", "pst_all.pdf"), corner_radius=1, font=self.font)
        pst_tree_button.grid(row=1, column=1, pady=(10, 10))

        pst_report_button = ctk.CTkButton(self.dynamic_frame, text="Report", command=lambda: self.open_file("pst", "report", "pst.multiqc.html"), corner_radius=1, font=self.font)
        pst_report_button.grid(row=2, column=1, pady=(10, 10))

    def open_file(self, type, subdir, filename):
        file_path = os.path.join(self.marpledir, "results", type, subdir, filename)
        if filename.endswith(".pdf"):
            print(f'xdg-open {file_path}')
            os.system(f"xdg-open {file_path}")
        elif filename.endswith(".html"):
            os.system(f"xdg-open {file_path}")
        

app = App()
app.mainloop()


sudo apt-get install default-jre libreoffice-java-common

In [6]:
import tkinter as tk
from tkinter import Menu, filedialog, ttk
import customtkinter as ctk
import os
import gzip
import shutil
import subprocess
from pathlib import Path
from PIL import Image
import threading

class App(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.colmode = "light"
        self.colswitch = "#eceff1"
        self.update_mode()

        self.marpledir = os.path.join(os.path.join(Path.home(), 'marple'))
        
        self.geometry("600x920")
        self.title("MARPLE")
        self.attributes('-alpha', 0.95)
        
        self.font = ctk.CTkFont(family="Helvetica", size=16)
        self.large_font = ctk.CTkFont(family="Helvetica", size=20)
        
        self.barcode_rows = []
        self.transfer_type = 'Pgt'
        self.minknow_dir = ""

        # Image setup
        logo_path = os.path.join(os.getcwd(), "MARPLE_logo.png")
        try:
            logo_image = ctk.CTkImage(Image.open(logo_path), size=(600, 200))
        except Exception as e:
            print(f"Error loading image: {e}")
            logo_image = None
        
        self.logo_label = ctk.CTkLabel(self, image=logo_image, text="", bg_color=self.colswitch)
        self.logo_label.pack(pady=(20, 10))

        # Menu setup (Sandwich Menu)
        self.menu_bar = Menu(self)
        self.config(menu=self.menu_bar)

        self.marple_menu = Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Menu", menu=self.marple_menu)
        self.marple_menu.add_command(label="Run MARPLE", command=self.show_home)
        self.marple_menu.add_command(label="Transfer Reads", command=self.show_transfer_reads)
        self.marple_menu.add_command(label="Results", command=self.show_results)
        self.marple_menu.add_command(label="About", command=self.show_about)
        self.marple_menu.add_separator()
        self.marple_menu.add_command(label="Exit", command=self.quit)

        # Light/Dark mode toggle in the menu
        self.theme_menu = Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="Settings", menu=self.theme_menu)
        self.theme_menu.add_command(label=f'{"Dark mode" if self.colmode == "light" else "Light mode"}', command=self.switch_mode)

        # Run MARPLE button (Home Page)
        self.run_marple_button = ctk.CTkButton(self, text="RUN MARPLE", command=self.run_marple, corner_radius=1, font=self.large_font)
        self.run_marple_button.pack(pady=(30, 30))
        
        self.stop_button = ctk.CTkButton(self, text="STOP MARPLE", command=self.stop_marple, corner_radius=1, font=self.large_font)
        self.stop_button.pack(pady=(10, 20))
        self.snakemake_process = None

        # Placeholder for dynamic content
        self.dynamic_frame = None

        # Progress bar
        self.progress_bar = ctk.CTkProgressBar(self, orientation="horizontal")
        
        # Notification frame
        self.notification_frame = ctk.CTkFrame(self, bg_color="gray10")

    def update_mode(self):
        ctk.set_appearance_mode(self.colmode)
        self.colswitch = "#eceff1" if self.colmode == "light" else "gray10"

    def switch_mode(self):
        self.colmode = "dark" if self.colmode == "light" else "light"
        self.update_mode()
        self.update_ui()
        # Update the theme toggle text in the menu
        self.theme_menu.entryconfig(0, label=f'{"Dark mode" if self.colmode == "light" else "Light mode"}')

    def update_ui(self):
        self.logo_label.configure(bg_color=self.colswitch)
        if self.dynamic_frame:
            self.dynamic_frame.configure(bg_color=self.colswitch)

    def show_home(self):
        self.clear_dynamic_frame()
        self.run_marple_button.pack(pady=(30, 30))
        self.stop_button.pack(pady=(10, 20))

    def show_transfer_reads(self):
        self.clear_dynamic_frame()
        self.dynamic_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        self.dynamic_frame.pack(fill="both", expand=True, pady=20)

        # Transfer reads options
        self.select_dir_button = ctk.CTkButton(self.dynamic_frame, command=self.select_experiment, text="Select MinKNOW Directory", corner_radius=1, font=self.font)
        self.select_dir_button.pack(pady=(20, 10))

        self.expname_label = ctk.CTkLabel(self.dynamic_frame, text="Experiment Name: ", corner_radius=1, font=self.font)
        self.expname_label.pack(pady=(10, 20))

        self.segmented_button = ctk.CTkSegmentedButton(self.dynamic_frame, values=['Pgt', 'Pst'], command=self.on_segmented_button_click, corner_radius=1, font=self.font)
        self.segmented_button.set('Pgt')
        self.segmented_button.pack(pady=(10, 20))

        self.add_row_button = ctk.CTkButton(self.dynamic_frame, text="Add Barcode Row", command=self.add_barcode_row, corner_radius=1, font=self.font)
        self.add_row_button.pack(pady=(10, 20))

        self.transfer_reads_button = ctk.CTkButton(self.dynamic_frame, command=self.transfer_reads, text="Transfer Reads", corner_radius=1, font=self.large_font)
        self.transfer_reads_button.pack(pady=(10, 20))

        # Create a frame with a scrollbar
        self.canvas = ctk.CTkCanvas(self.dynamic_frame, bg=self.colswitch)
        self.scrollbar = ttk.Scrollbar(self.dynamic_frame, orient="vertical", command=self.canvas.yview)
        self.scrollable_frame = ctk.CTkFrame(self.canvas, bg_color=self.colswitch, corner_radius=1)

        # Configure the canvas and scrollbar
        self.scrollable_frame.bind("<Configure>", lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))
        self.canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")
        self.canvas.configure(yscrollcommand=self.scrollbar.set)

        # Limit the height of the canvas (5 rows height)
        self.canvas.pack(side="left", fill="both", expand=True, padx=(5, 0), pady=20)
        self.scrollbar.pack(side="right", fill="y")

    def add_barcode_row(self):
        row = ctk.CTkFrame(self.scrollable_frame, bg_color=self.colswitch, corner_radius=1)
        row.pack(fill="x", padx=5, pady=2)
        row.grid_columnconfigure(0, weight=1)

        barcode_label = ctk.CTkLabel(row, text="Barcode:", font=self.font)
        barcode_label.pack(side="left", padx=5)

        barcode_entry = ctk.CTkEntry(row, width=60, corner_radius=1, font=self.font)
        barcode_entry.pack(side="left", padx=5)

        sample_label = ctk.CTkLabel(row, text="Sample Name:", font=self.font)
        sample_label.pack(side="left", padx=5)

        sample_entry = ctk.CTkEntry(row, width=200, corner_radius=1, font=self.font)
        sample_entry.pack(side="left", padx=5)

        self.barcode_rows.append((barcode_entry, sample_entry))
        
        # Enable scrolling if more than 5 rows
        if len(self.barcode_rows) > 5:
            self.canvas.configure(height=5 * 40)  # Each row is about 40 pixels high

    def show_about(self):
        self.clear_dynamic_frame()
        self.dynamic_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        self.dynamic_frame.pack(fill="both", expand=True, pady=20)

        about_label = ctk.CTkLabel(self.dynamic_frame, text="MARPLE Diagnostics:\npoint-of-care, strain-level disease diagnostics and\nsurveillance tool for complex fungal pathogens\n\nVersion: 2.0-alpha", font=self.large_font)
        about_label.pack(pady=(20, 20))

        devs_label = ctk.CTkLabel(self.dynamic_frame, text="Developers:\nLoizos Savva\nAnthony Bryan\nGuru V. Radhakrishnan")
        devs_label.pack(pady=(20, 20))
        
        copyright_label = ctk.CTkLabel(self.dynamic_frame, text="© 2024 Saunders Lab")
        copyright_label.pack()

    def clear_dynamic_frame(self):
        if self.dynamic_frame:
            self.dynamic_frame.destroy()
        self.run_marple_button.pack_forget()
        self.stop_button.pack_forget()

    def on_segmented_button_click(self, choice):
        self.transfer_type = choice

    def select_experiment(self):
        default_dir = "/var/lib/minknow/data/"
        self.minknow_dir = filedialog.askdirectory(initialdir=default_dir)
        if self.minknow_dir:
            expname = os.path.basename(self.minknow_dir)
            self.expname_label.configure(text=f"Experiment Name: {expname}")

    def transfer_reads(self):
        if not self.minknow_dir:
            print("MinKNOW directory not selected.")
            return

        # Start the progress bar
        self.progress_bar.pack(pady=(10, 20))
        self.progress_bar.configure(mode="indeterminate")
        self.progress_bar.start()

        # Run the task in a separate thread to avoid freezing the UI
        thread = threading.Thread(target=self.process_reads)
        thread.start()

    def process_reads(self):
        try:
            total_barcodes = len(self.barcode_rows)
            if total_barcodes == 0:
                self.printin("No barcodes to process.")
                self.progress_bar.stop()
                self.progress_bar.pack_forget()
                return

            for barcode_entry, sample_entry in self.barcode_rows:
                barcode = format(int(barcode_entry.get().strip()), '02d')
                sample = sample_entry.get().strip()

                if not barcode:
                    continue  # Skip empty barcode entries

                for root, dirs, files in os.walk(self.minknow_dir):
                    for dir in dirs:
                        if dir == 'pass':
                            barcode_dir = os.path.join(root, dir, f'barcode{barcode}')
                            try:
                                output_file = os.path.join(self.marpledir, 'reads', self.transfer_type.lower(), f'{sample}.fastq.gz')
                                with gzip.open(output_file, 'wb') as fout:
                                    for file in os.listdir(barcode_dir):
                                        if file.endswith(".gz"):
                                            with gzip.open(os.path.join(barcode_dir, file), 'rb') as f:
                                                reads = f.readlines()
                                                fout.writelines(reads)
                                        elif file.endswith(".fastq"):
                                            with open(os.path.join(barcode_dir, file), 'rb') as f:
                                                reads = f.readlines()
                                                fout.writelines(reads)
                                self.printin(f"Successfully transferred reads for barcode{barcode} to {output_file}")
                            except Exception as e:
                                self.printin(f"An error occurred while processing barcode{barcode}: {e}")
        finally:
            self.progress_bar.stop()
            self.progress_bar.pack_forget()

    def run_marple(self):
        # Start progress bar
        self.progress_bar.pack(pady=(10, 20))
        self.progress_bar.configure(mode="indeterminate")
        self.progress_bar.start()

        # Run the Snakemake process in a separate thread
        thread = threading.Thread(target=self.run_snakemake)
        thread.start()

    def run_snakemake(self):
        if shutil.which("mamba") is not None:
            try:
                env_list = subprocess.check_output(["mamba", "env", "list"], text=True)
                if "marple-env" in env_list:
                    self.stop_marple()
                    self.start_marple()
                else:
                    self.printin("mamba environment marple-env not found.")
            except subprocess.CalledProcessError as e:
                self.printin(f"Error running command: {e}")
        else:
            self.printin("mamba not found.")
        
        self.progress_bar.stop()
        self.progress_bar.pack_forget()

    def start_marple(self):
        log_file = "snakemake.log"
        with open(log_file, "w") as log:
            self.snakemake_process = subprocess.Popen(
                ["mamba", "run", "-n", "marple-env", "snakemake", "--cores", "all", "--rerun-incomplete"],
                cwd=self.marpledir, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1
            )
            self.display_output(self.snakemake_process)

    def stop_marple(self):
        # if self.snakemake_process and self.snakemake_process.poll() is None:
        self.snakemake_process.terminate()
        self.printin("MARPLE process terminated.")
        self.unlock_snakemake()

    def unlock_snakemake(self):
        try:
            subprocess.run(["mamba", "run", "-n", "marple-env", "snakemake", "--unlock"], cwd=self.marpledir)
            self.printin("Snakemake unlocked.")
        except Exception as e:
            self.printin(f"Failed to unlock Snakemake: {e}")

    def display_output(self, process):
        if not hasattr(self, 'output_text'):
            self.output_text = ctk.CTkTextbox(self, width=600, height=400)
            self.output_text.pack(pady=(20, 20))
            self.output_text.configure(state="disabled")

        self.output_text.configure(state="normal")
        self.output_text.insert("end", process.stdout.read())
        self.output_text.see("end")

        # for line in process.stdout.readline():
        #     if line:
        #         self.output_text.insert("end", line)
        #         self.output_text.see("end")
        #     self.update()

        process.stdout.close()
        process.wait()

        if process.returncode != 0:
            self.output_text.insert("end", f"MARPLE run failed with exit code {process.returncode}. Error: {process.stderr.read()}")
            self.output_text.see("end")
        else:
            self.output_text.insert("end", "MARPLE run completed successfully.")
            self.output_text.see("end")

        self.output_text.configure(state="disabled")  # Disable the text box again

    def toggle_output_display(self):
        if self.output_text.winfo_viewable():
            self.output_text.pack_forget()
        else:
            self.output_text.pack(pady=(20, 20))
    
    def check_process(self, process, log_file):
        if process.poll() is None:
            self.after(100, self.check_process, process, log_file)
        else:
            self.progress_bar.stop()
            self.progress_bar.pack_forget()
            status = process.returncode
            if status != 0:
                self.printin(f"Command failed with exit status {status}. Error log:")
                with open(log_file, "r") as log:
                    self.printin(log.read())
            else:
                self.printin("MARPLE run completed successfully.")

    def printin(self, stdout):
        self.notification_frame.pack(side="bottom", fill="x", expand="True")
        for widget in self.notification_frame.winfo_children():
            widget.destroy()
        
        label = ctk.CTkLabel(self.notification_frame, text=stdout, bg_color=self.colswitch)
        label.pack()
        
        self.after(5000, self.clear_notification)
    
    def clear_notification(self):
        for widget in self.notification_frame.winfo_children():
            widget.destroy()
        self.notification_frame.pack_forget()

    def show_results(self):
        self.clear_dynamic_frame()
        self.dynamic_frame = ctk.CTkFrame(self, bg_color=self.colswitch)
        self.dynamic_frame.pack(fill="both", expand=True, pady=20)

        # Two columns
        self.dynamic_frame.grid_columnconfigure(0, weight=1)
        self.dynamic_frame.grid_columnconfigure(1, weight=1)

        # Pgt Column
        pgt_label = ctk.CTkLabel(self.dynamic_frame, text="Pgt", font=self.large_font, anchor="center")
        pgt_label.grid(row=0, column=0, pady=(10, 10))

        pgt_tree_button = ctk.CTkButton(self.dynamic_frame, text="Tree", command=lambda: self.open_file("pgt", "trees", "pgt_all.pdf"), corner_radius=1, font=self.font)
        pgt_tree_button.grid(row=1, column=0, pady=(10, 10))

        pgt_report_button = ctk.CTkButton(self.dynamic_frame, text="Report", command=lambda: self.open_file("pgt", "report", "pgt.multiqc.html"), corner_radius=1, font=self.font)
        pgt_report_button.grid(row=2, column=0, pady=(10, 10))

        # Pst Column
        pst_label = ctk.CTkLabel(self.dynamic_frame, text="Pst", font=self.large_font, anchor="center")
        pst_label.grid(row=0, column=1, pady=(10, 10))

        pst_tree_button = ctk.CTkButton(self.dynamic_frame, text="Tree", command=lambda: self.open_file("pst", "trees", "pst_all.pdf"), corner_radius=1, font=self.font)
        pst_tree_button.grid(row=1, column=1, pady=(10, 10))

        pst_report_button = ctk.CTkButton(self.dynamic_frame, text="Report", command=lambda: self.open_file("pst", "report", "pst.multiqc.html"), corner_radius=1, font=self.font)
        pst_report_button.grid(row=2, column=1, pady=(10, 10))

    def open_file(self, type, subdir, filename):
        file_path = os.path.join(self.marpledir, "results", type, subdir, filename)
        if filename.endswith(".pdf"):
            print(f'xdg-open {file_path}')
            os.system(f"xdg-open {file_path}")
        elif filename.endswith(".html"):
            os.system(f"xdg-open {file_path}")
        

app = App()
app.mainloop()

invalid command name "134841975083904<lambda>"
    while executing
"134841975083904<lambda>"
    ("after" script)
Exception in thread Thread-36 (run_snakemake):
Traceback (most recent call last):
  File "/home/marple/micromamba/envs/marple-env/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/marple/micromamba/envs/marple-env/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/home/marple/micromamba/envs/marple-env/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_288623/1445235295.py", line 256, in run_snakemake
  File "/tmp/ipykernel_288623/1445235295.py", line 279, in stop_marple
AttributeError: 'NoneType' object has no attribute 'terminate'
Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/marple/micromamba/envs/marple-env/lib/python3.11/tkinter/__init__.py", line 1967, in __call__
 